<a href="https://colab.research.google.com/github/sindhugovi/Intrusion_Detection_HLS/blob/main/intrusion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing libraries

In [ ]:
import pandas as pd
import numpy as np


loading the dataset as pandas dataframe

In [ ]:
data=pd.read_csv("IoT_Intrusion.csv")

printing the first 5 rows

In [ ]:
data.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.00,6.00,64.00,0.329807,0.329807,0.0,1,0,1,...,0.000000,54.00,83343831.92,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.000000,57.04,6.33,64.00,4.290556,4.290556,0.0,0,0,0,...,2.822973,57.04,82926067.47,9.5,10.464666,4.010353,160.987842,0.05,141.55,DoS-TCP_Flood
2,0.000000,0.00,1.00,64.00,33.396799,33.396799,0.0,0,0,0,...,0.000000,42.00,83127993.93,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
3,0.328175,76175.00,17.00,64.00,4642.133010,4642.133010,0.0,0,0,0,...,0.000000,50.00,83015696.38,9.5,10.000000,0.000000,0.000000,0.00,141.55,DoS-UDP_Flood
4,0.117320,101.73,6.11,65.91,6.202211,6.202211,0.0,0,1,0,...,23.113111,57.88,82972999.18,9.5,11.346876,32.716243,3016.808286,0.19,141.55,DoS-SYN_Flood


information about the dataset

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 47 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   flow_duration    1048575 non-null  float64
 1   Header_Length    1048575 non-null  float64
 2   Protocol Type    1048575 non-null  float64
 3   Duration         1048575 non-null  float64
 4   Rate             1048575 non-null  float64
 5   Srate            1048575 non-null  float64
 6   Drate            1048575 non-null  float64
 7   fin_flag_number  1048575 non-null  int64  
 8   syn_flag_number  1048575 non-null  int64  
 9   rst_flag_number  1048575 non-null  int64  
 10  psh_flag_number  1048575 non-null  int64  
 11  ack_flag_number  1048575 non-null  int64  
 12  ece_flag_number  1048575 non-null  int64  
 13  cwr_flag_number  1048575 non-null  int64  
 14  ack_count        1048575 non-null  float64
 15  syn_count        1048575 non-null  float64
 16  fin_count        1

To get the unique values of labels

In [ ]:
col_46_values = data.iloc[:, 46]

In [ ]:
unique_values = col_46_values.unique()

# Count the number of unique values
num_unique = col_46_values.nunique()

print("Unique values in column 46:", unique_values)
print("Number of unique values:", num_unique)

Unique values in column 46: ['DDoS-RSTFINFlood' 'DoS-TCP_Flood' 'DDoS-ICMP_Flood' 'DoS-UDP_Flood'
 'DoS-SYN_Flood' 'Mirai-greeth_flood' 'DDoS-SynonymousIP_Flood'
 'Mirai-udpplain' 'DDoS-SYN_Flood' 'DDoS-PSHACK_Flood' 'DDoS-TCP_Flood'
 'DDoS-UDP_Flood' 'BenignTraffic' 'MITM-ArpSpoofing'
 'DDoS-ACK_Fragmentation' 'Mirai-greip_flood' 'DoS-HTTP_Flood'
 'DDoS-ICMP_Fragmentation' 'Recon-PortScan' 'DNS_Spoofing'
 'DDoS-UDP_Fragmentation' 'Recon-OSScan' 'XSS' 'DDoS-HTTP_Flood'
 'Recon-HostDiscovery' 'CommandInjection' 'VulnerabilityScan'
 'DDoS-SlowLoris' 'Backdoor_Malware' 'BrowserHijacking'
 'DictionaryBruteForce' 'SqlInjection' 'Recon-PingSweep'
 'Uploading_Attack']
Number of unique values: 34


splitting the datas into test and train sets and
encoding label as number

In [ ]:
import pandas as pd
import numpy as np


# Convert DataFrame to NumPy array
data = np.array(data)
m, n = data.shape

# Shuffle the rows
np.random.shuffle(data)

# Split and transpose
data_dev = data[0:10000].T
data_train = data[10000:m].T

# Extract labels and features
Y_dev = data_dev[46]
X_dev = data_dev[0:46].astype(float)  # Convert features to float

Y_train = data_train[46]
X_train = data_train[0:46].astype(float)

# Encode labels consistently
all_labels = np.concatenate((Y_train, Y_dev))
classes, _ = np.unique(all_labels, return_inverse=True)
label_to_index = {label: idx for idx, label in enumerate(classes)}

Y_train_encoded = np.vectorize(label_to_index.get)(Y_train).reshape(1, -1)
Y_dev_encoded = np.vectorize(label_to_index.get)(Y_dev).reshape(1, -1)

# Normalize features using training set statistics (min-max scaling)
min_vals = X_train.min(axis=1, keepdims=True)
max_vals = X_train.max(axis=1, keepdims=True)

X_train = (X_train - min_vals) / (max_vals - min_vals + 1e-8)
X_dev = (X_dev - min_vals) / (max_vals - min_vals + 1e-8)

# Confirm shapes
_, m_train = X_train.shape
_, m_dev = X_dev.shape

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train_encoded.shape}")
print(f"X_dev shape: {X_dev.shape}, Y_dev shape: {Y_dev_encoded.shape}")
print(f"Number of classes: {len(classes)}")
print("Classes:", classes)


X_train shape: (46, 1038575), Y_train shape: (1, 1038575)
X_dev shape: (46, 10000), Y_dev shape: (1, 10000)
Number of classes: 34
Classes: ['Backdoor_Malware' 'BenignTraffic' 'BrowserHijacking' 'CommandInjection'
 'DDoS-ACK_Fragmentation' 'DDoS-HTTP_Flood' 'DDoS-ICMP_Flood'
 'DDoS-ICMP_Fragmentation' 'DDoS-PSHACK_Flood' 'DDoS-RSTFINFlood'
 'DDoS-SYN_Flood' 'DDoS-SlowLoris' 'DDoS-SynonymousIP_Flood'
 'DDoS-TCP_Flood' 'DDoS-UDP_Flood' 'DDoS-UDP_Fragmentation' 'DNS_Spoofing'
 'DictionaryBruteForce' 'DoS-HTTP_Flood' 'DoS-SYN_Flood' 'DoS-TCP_Flood'
 'DoS-UDP_Flood' 'MITM-ArpSpoofing' 'Mirai-greeth_flood'
 'Mirai-greip_flood' 'Mirai-udpplain' 'Recon-HostDiscovery' 'Recon-OSScan'
 'Recon-PingSweep' 'Recon-PortScan' 'SqlInjection' 'Uploading_Attack'
 'VulnerabilityScan' 'XSS']


In [ ]:
import numpy as np

# Define number of output classes and input features
num_classes = 34
input_size = 46

intializing weights and bias with some random numbers

In [ ]:
def init_params():
    W1 = np.random.randn(128, input_size) * np.sqrt(2 / input_size)
    b1 = np.zeros((128, 1))

    W2 = np.random.randn(64, 128) * np.sqrt(2 / 128)
    b2 = np.zeros((64, 1))

    W3 = np.random.randn(32, 64) * np.sqrt(2 / 64)
    b3 = np.zeros((32, 1))

    W4 = np.random.randn(num_classes, 32) * np.sqrt(2 / 32)
    b4 = np.zeros((num_classes, 1))

    return W1, b1, W2, b2, W3, b3, W4, b4


defining the functions for neural network

In [ ]:
def ReLU(Z):
    return np.maximum(0, Z)

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    Z_shifted = Z - np.max(Z, axis=0, keepdims=True)
    expZ = np.exp(Z_shifted)
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def one_hot(Y):
    one_hot_Y = np.zeros((num_classes, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y


The forward propogation function

In [ ]:
def forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X):
    Z1 = W1 @ X + b1
    A1 = ReLU(Z1)

    Z2 = W2 @ A1 + b2
    A2 = ReLU(Z2)

    Z3 = W3 @ A2 + b3
    A3 = ReLU(Z3)

    Z4 = W4 @ A3 + b4
    A4 = softmax(Z4)

    return Z1, A1, Z2, A2, Z3, A3, Z4, A4


the backward propagation function

In [ ]:
def backward_prop(Z1, A1, Z2, A2, Z3, A3, Z4, A4,
                  W1, W2, W3, W4, X, Y):
    m = X.shape[1]
    one_hot_Y = one_hot(Y)

    dZ4 = A4 - one_hot_Y
    dW4 = (1/m) * dZ4 @ A3.T
    db4 = (1/m) * np.sum(dZ4, axis=1, keepdims=True)

    dZ3 = W4.T @ dZ4 * ReLU_deriv(Z3)
    dW3 = (1/m) * dZ3 @ A2.T
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

    dZ2 = W3.T @ dZ3 * ReLU_deriv(Z2)
    dW2 = (1/m) * dZ2 @ A1.T
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T @ dZ2 * ReLU_deriv(Z1)
    dW1 = (1/m) * dZ1 @ X.T
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3, dW4, db4


the function for updating weights and parameters

In [ ]:
def update_params(W1, b1, W2, b2, W3, b3, W4, b4,
                  dW1, db1, dW2, db2, dW3, db3, dW4, db4, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1

    W2 -= alpha * dW2
    b2 -= alpha * db2

    W3 -= alpha * dW3
    b3 -= alpha * db3

    W4 -= alpha * dW4
    b4 -= alpha * db4

    return W1, b1, W2, b2, W3, b3, W4, b4


the function to find prediction and accuracy

In [ ]:
def get_predictions(A):
    return np.argmax(A, axis=0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)


training funtion

In [ ]:
def gradient_descent(X, Y, alpha=0.1, iterations=200):
    W1, b1, W2, b2, W3, b3, W4, b4 = init_params()

    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3, Z4, A4 = forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X)
        dW1, db1, dW2, db2, dW3, db3, dW4, db4 = backward_prop(
            Z1, A1, Z2, A2, Z3, A3, Z4, A4,
            W1, W2, W3, W4, X, Y
        )
        W1, b1, W2, b2, W3, b3, W4, b4 = update_params(
            W1, b1, W2, b2, W3, b3, W4, b4,
            dW1, db1, dW2, db2, dW3, db3, dW4, db4,
            alpha
        )

        if i % 10 == 0:
            predictions = get_predictions(A4)
            acc = get_accuracy(predictions, Y)
            print(f"Iteration {i}: Accuracy = {acc:.4f}")

    return W1, b1, W2, b2, W3, b3, W4, b4


In [ ]:
# Train the model
W1, b1, W2, b2, W3, b3, W4, b4 = gradient_descent(X_train, Y_train_encoded[0], alpha=0.1, iterations=500)


Iteration 0: Accuracy = 0.0137
Iteration 10: Accuracy = 0.3269
Iteration 20: Accuracy = 0.6255
Iteration 30: Accuracy = 0.6262
Iteration 40: Accuracy = 0.6262
Iteration 50: Accuracy = 0.6300
Iteration 60: Accuracy = 0.6517
Iteration 70: Accuracy = 0.6600
Iteration 80: Accuracy = 0.6629
Iteration 90: Accuracy = 0.6648
Iteration 100: Accuracy = 0.6727
Iteration 110: Accuracy = 0.6781
Iteration 120: Accuracy = 0.6814
Iteration 130: Accuracy = 0.6844
Iteration 140: Accuracy = 0.6979
Iteration 150: Accuracy = 0.7094
Iteration 160: Accuracy = 0.7124
Iteration 170: Accuracy = 0.7269
Iteration 180: Accuracy = 0.6765
Iteration 190: Accuracy = 0.6774
Iteration 200: Accuracy = 0.6711
Iteration 210: Accuracy = 0.6714
Iteration 220: Accuracy = 0.6715
Iteration 230: Accuracy = 0.6716
Iteration 240: Accuracy = 0.6722
Iteration 250: Accuracy = 0.6724
Iteration 260: Accuracy = 0.6726
Iteration 270: Accuracy = 0.6728
Iteration 280: Accuracy = 0.6740
Iteration 290: Accuracy = 0.6821
Iteration 300: Accura

saving the quantized weights

In [ ]:
import numpy as np

# Quantize weights and biases to int32 using scale (e.g., Q16.16 format)
def quantize_to_int32(arr, scale=1e6):
    return np.clip(np.round(arr * scale), -2**31 + 1, 2**31 - 1).astype(np.int32)

def quantize_bias_to_int32(bias, scale=1e6):
    return np.clip(np.round(bias.reshape(-1) * scale), -2**31 + 1, 2**31 - 1).astype(np.int32)


In [ ]:
def save_as_header_int32(var, name, filename):
    with open(filename, "w") as f:
        f.write(f"#ifndef {name.upper()}_H\n#define {name.upper()}_H\n\n")
        f.write("#include <stdint.h>\n\n")
        f.write(f"const int32_t {name}[{var.shape[0]}][{var.shape[1]}] = {{\n")
        for row in var:
            f.write("  { " + ", ".join(str(x) for x in row) + " },\n")
        f.write("};\n\n#endif\n")

def save_bias_as_header_int32(bias, name, filename):
    with open(filename, "w") as f:
        f.write(f"#ifndef {name.upper()}_H\n#define {name.upper()}_H\n\n")
        f.write("#include <stdint.h>\n\n")
        f.write(f"const int32_t {name}[{len(bias)}] = {{\n  ")
        f.write(", ".join(str(b) for b in bias))
        f.write("\n};\n\n#endif\n")


In [ ]:
# After training: save quantized versions
scale = 1e6  # You can tune this value (e.g., 1e5 or 1e7 based on accuracy vs. range)

# Quantize weights
W1_q = quantize_to_int32(W1, scale)
W2_q = quantize_to_int32(W2, scale)
W3_q = quantize_to_int32(W3, scale)
W4_q = quantize_to_int32(W4, scale)

# Quantize biases
b1_q = quantize_bias_to_int32(b1, scale)
b2_q = quantize_bias_to_int32(b2, scale)
b3_q = quantize_bias_to_int32(b3, scale)
b4_q = quantize_bias_to_int32(b4, scale)

# Save headers
save_as_header_int32(W1_q, "W1", "W1.h")
save_as_header_int32(W2_q, "W2", "W2.h")
save_as_header_int32(W3_q, "W3", "W3.h")
save_as_header_int32(W4_q, "W4", "W4.h")

save_bias_as_header_int32(b1_q, "b1", "b1.h")
save_bias_as_header_int32(b2_q, "b2", "b2.h")
save_bias_as_header_int32(b3_q, "b3", "b3.h")
save_bias_as_header_int32(b4_q, "b4", "b4.h")


In [ ]:
W1[0][0]

np.float64(-0.4038683200503629)

In [ ]:
W1_q[0][0]

np.int32(-403868)

exporting the test image as header file

In [ ]:
def export_test_sample_fixed32(X_dev, index, filename, scale=1e6, var_name="sample_input"):
    """
    Quantizes and exports a single test input sample as a fixed-point int32 array.

    Parameters:
        X_dev (ndarray): Dev/test features (46 x m).
        index (int): Index of the test sample.
        filename (str): Output .h filename.
        scale (float): Fixed-point scaling factor (e.g., 1e6).
        var_name (str): Name of the C array variable.
    """
    sample = X_dev[:, index]                     # shape: (46,)
    quantized = np.clip(np.round(sample * scale), -2**31 + 1, 2**31 - 1).astype(np.int32)

    with open(filename, "w") as f:
        f.write(f"#ifndef TEST_SAMPLE_H\n#define TEST_SAMPLE_H\n\n")
        f.write("#include <stdint.h>\n\n")
        f.write(f"const int32_t {var_name}[46] = {{\n    ")
        f.write(", ".join(str(val) for val in quantized))
        f.write("\n};\n\n#endif // TEST_SAMPLE_H\n")


In [ ]:
# Example: Export the 10th test sample (X_dev must be available from your pipeline)
export_test_sample_fixed32(X_dev, index=300, filename="test_sample.h", scale=1e6)


for checking the results of hls

In [ ]:
index = 300 # or whatever index you used in export_test_sample
print("Encoded label:", Y_dev_encoded[0, index])
print("Original label:", classes[Y_dev_encoded[0, index]])


Encoded label: 14
Original label: DDoS-UDP_Flood
